<a href="https://colab.research.google.com/github/TheRadDani/logistic-regression/blob/main/Naive_Bayesynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
# import config
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy.stats as st
from pandas_profiling import ProfileReport
register_matplotlib_converters()
sns.set(style="whitegrid", palette= "muted", font_scale =1.5)
rcParams['figure.figsize']=22,10

In [ ]:
import numpy as np
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix
from pyspark.sql import SparkSession
import pandas as pd
sc = SparkSession.builder\
     .master("local")\
     .appName("Word Count")\
     .config("spark.some.config.option", "some-value")\
     .getOrCreate()
sqlContext = SQLContext(sc)
file = "train.csv"
test_size = .3
df = pd.read_csv(file)
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
target = df.Survived

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df.Sex)
list(le.classes_)
encoded = le.transform(df.Sex)
df.Sex = encoded
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
imputed = imputer.fit_transform(df['Age'].values.reshape(-1,1))
df.Age = imputed
df.rename(columns={"Survived":"label"},inplace=True)
data = sqlContext.createDataFrame(df)
inputs = df.drop("label",axis=1)
features = list(inputs.columns)

va = VectorAssembler(inputCols = features, outputCol='features')

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(inputs,target,test_size=test_size,random_state=42,shuffle=True)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model = model.fit(X_train,y_train)

va_df = va.transform(data)
va_df = va_df.select(['features', 'label'])
#va_df.show()
(train, test) = va_df.randomSplit([0.8, 0.2])
nb = NaiveBayes(smoothing=1.0, modelType="gaussian")
#nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
nb = nb.fit(train)
pred = nb.transform(test)
pred.show(3) 
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction")
acc = evaluator.evaluate(pred)
 
print("Prediction Accuracy: ", acc)
 
y_pred=pred.select("prediction").collect()
y_orig=pred.select("label").collect()

cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[1.0,0.0,14.0,120.0]|    1|[-15.572706606307...|[0.00116787615562...|       1.0|
|[1.0,0.0,30.0,56....|    1|[-10.556499263703...|[0.05407298915068...|       1.0|
|[1.0,0.0,31.0,113...|    1|[-14.070505853903...|[0.00281818775186...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 3 rows

Prediction Accuracy:  0.790715448033548
Confusion Matrix:
[[92 14]
 [23 50]]
